In [84]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import urlparse, parse_qs, urlunparse

from bs4 import BeautifulSoup

import re
import csv
import time
import math
import pandas as pd
import unicodedata

import arxiv

In [96]:
# Useful functions

def prepare_base_url(full_url):
    parsed_url = urlparse(full_url)
    query_params = parse_qs(parsed_url.query)
    query_params.pop('page', None)

    # Reconstruct the query string, ensuring spaces are encoded correctly
    query_string = "&".join(f"{key}={value[0].replace(' ', '%20')}" for key, value in query_params.items())
    new_parsed_url = parsed_url._replace(query=query_string)
    
    return urlunparse(new_parsed_url)

def get_total_pages_from_inspire(driver, full_query_url):
    # Prepare the base URL by removing the 'page' parameter
    base_url = prepare_base_url(full_query_url)
    print(base_url)
    
    # Fetch the first page to get the total number of results
    driver.get(base_url + "&page=1")
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for the page to load
    time.sleep(2)  # Adjust this time based on your network speed

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the total number of results
    results_span = soup.find('span', string=re.compile(r'\d+\sresults'))
    total_results = int(results_span.text.split()[0].replace(',', '')) if results_span else 0
    
    # Extract the results per page from the URL
    match = re.search(r"&size=(\d+)&", full_query_url)
    results_per_page = int(match.group(1)) if match else 25  # Default to 25 if not found
    
    # Calculate the total number of pages
    total_pages = math.ceil(total_results / results_per_page)
    
    return total_pages

# Function to extract the number of papers per page
def get_papers_per_page(url):
    # Regular expression to find the size parameter
    match = re.search(r"&size=(\d+)&", url)

    return int(match.group(1))

# Function to extract DOI
def extract_doi(entry):
    doi_pattern = re.compile(r'(10\.\d{4,9}/[-._;()/:A-Za-z0-9]+)')
    doi = None
    doi_tag = entry.find('a', href=doi_pattern)
    if doi_tag:
        doi_match = doi_pattern.search(doi_tag['href'])
        doi = doi_match.group(0) if doi_match else None
    return doi

# Function to extract ePrint number
def extract_eprint_number(entry):
    eprint_number = None
    eprint_section = entry.find(lambda tag: tag.name == 'a' and 'arxiv.org' in tag.get('href', ''))
    if eprint_section:
        eprint_number = eprint_section.get_text(strip=True)
    return eprint_number

# Function to extract arXiv category
def extract_arxiv_category_v2(entry):
    arxiv_category = None
    eprint_link = entry.find(lambda tag: tag.name == 'a' and 'arxiv.org' in tag.get('href', ''))
    if eprint_link:
        category_span = eprint_link.find_next_sibling('span')
        if category_span:
            arxiv_category = category_span.get_text(strip=True).strip('[]')
    return arxiv_category

# Revised function to extract citation counts based on the demonstrated algorithm
def extract_citation_count_v2(entry):
    citation_tag = entry.find_next(lambda tag: tag.name == 'span' and ('citation' in tag.text or 'citations' in tag.text))
    if citation_tag:
        # Remove commas from the string before converting to int
        citation_count = int(citation_tag.text.split()[0].replace(',', ''))
        return citation_count
    return 0

# Function to extract paper details including citation counts
def extract_paper_details_v9(entry):
    #title_tag = entry.find('span', {'data-test-id': 'literature-detail-title'})
    #title = title_tag.get_text(strip=True) if title_tag else None
    
    title = extract_title_with_latex(entry)
    
    authors_tags = entry.find_all('a', {'data-test-id': 'author-link'})
    authors = ', '.join([tag.get_text(strip=True) for tag in authors_tags])

    eprint_number = extract_eprint_number(entry)
    arxiv_category = extract_arxiv_category_v2(entry)
    doi = extract_doi(entry)
    citation_count = extract_citation_count_v2(entry)

    return {
        'title': title,
        'authors': authors,
        'eprint_number': eprint_number,
        'arXiv_category': arxiv_category,
        'doi': doi,
        'citation_count': citation_count
    }

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('ASCII')

def extract_authors(entry):
    authors_tags = entry.find_all('a', {'data-test-id': 'author-link'})
    authors_list = [remove_accents(tag.get_text(strip=True)) for tag in authors_tags]
    return ", ".join(authors_list)

# Function to filter out duplicate and invalid entries
def filter_valid_entries(entries):
    filtered_entries = []
    seen_titles = set()
    for entry in entries:
        title = entry.find('span', {'data-test-id': 'literature-detail-title'})
        authors = entry.find_all('a', {'data-test-id': 'author-link'})
        if title and authors and title.get_text(strip=True) not in seen_titles:
            seen_titles.add(title.get_text(strip=True))
            filtered_entries.append(entry)
    return filtered_entries

def fetch_abstract(eprint_number):
    if not eprint_number:
        return "Abstract not found"
    try:
        client = arxiv.Client()
        search = arxiv.Search(id_list=[eprint_number])
        for result in client.results(search):
            abstract = result.summary.replace('\n', ' ')
            return abstract
    except Exception as e:
        return f"Error fetching abstract: {str(e)}"
    return "Abstract not found"


In [81]:
from bs4 import BeautifulSoup, NavigableString

def extract_title_with_latex(entry):
    title_element = entry.find('span', {'data-test-id': 'literature-detail-title'})
    if not title_element:
        return None

    def extract_text_latex(el):
        text_parts = []
        for content in el.children:
            if isinstance(content, NavigableString):
                current_text = str(content)
                if text_parts:
                    # Check and adjust space at the interface
                    if text_parts[-1].endswith(" ") and current_text.startswith(" "):
                        current_text = current_text.lstrip()  # Remove leading space
                    elif not text_parts[-1].endswith(" ") and not current_text.startswith(" "):
                        current_text = " " + current_text  # Add leading space
                text_parts.append(current_text)
            elif content.name == 'span' and 'katex' in content.get('class', []):
                # Extract LaTeX content
                latex_annotation = content.find('annotation', encoding='application/x-tex')
                if latex_annotation:
                    latex_text = latex_annotation.get_text()
                    # Check and adjust space at the interface
                    if text_parts and not text_parts[-1].endswith(" "):
                        latex_text = " " + latex_text
                    if not latex_text.endswith(" "):
                        latex_text += " "
                    text_parts.append(latex_text)
            elif content.name:
                # Recursively process other elements
                text_parts.append(extract_text_latex(content))
        return ''.join(text_parts)

    return extract_text_latex(title_element)

In [3]:
# Set up the WebDriver for Safari
driver = webdriver.Safari()


In [86]:
def scrape_page_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all entries on the page
    entries = soup.find_all(lambda tag: tag.name == 'div' and 'literature' in tag.attrs.get('data-test-id', ''))#soup.find_all('div', {'data-test-id': 'literature-results-item'})
    filtered_entries = filter_valid_entries(entries)
    
    # Lists to store data
    titles = []
    authors = []
    eprint_numbers = []
    arxiv_categories = []
    dois = []
    citation_counts = []
    
    # Extract details from each entry
    for entry in filtered_entries:
        
        title = extract_title_with_latex(entry)
        titles.append(title)

        # Extract authors using the new function
        authors_names = extract_authors(entry)
        authors.append(authors_names)
        
        # Extract eprint number
        eprint_number = extract_eprint_number(entry)
        arxiv_category = extract_arxiv_category_v2(entry)
        
        # Extract DOI
        doi = extract_doi(entry)
        citation_count = extract_citation_count_v2(entry)

        eprint_numbers.append(eprint_number)
        arxiv_categories.append(arxiv_category)
        dois.append(doi)
        citation_counts.append(citation_count)

    # Create a DataFrame from the extracted data
    data = pd.DataFrame({
        'Title': titles,
        'Authors': authors,
        'ePrint Number': eprint_numbers,
        'arXiv Category': arxiv_categories,
        'DOI': dois,
        'Citation Count': citation_counts
    })

    return data

def scrape_all_pages(driver, base_url, total_pages):
    all_data = pd.DataFrame()

    for page in range(1, total_pages + 1):
        # Replace or add the 'page' parameter in the URL
        page_url = re.sub(r"(page=\d+)", f"page={page}", base_url)
        if "page=" not in page_url:
            page_url += f"&page={page}"
            
        driver.get(page_url)
        
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load. Adjust the wait time as needed.
        time.sleep(3)  # Example: using a static wait time

        html_content = driver.page_source
        #print(html_content)
        
        # Scrape the data from the current page
        page_data = scrape_page_data(html_content)

        # Concatenate the data from this page to the overall data
        all_data = pd.concat([all_data, page_data], ignore_index=True)

    return all_data


In [29]:
# URL to scrape
url = "https://inspirehep.net/literature?sort=mostrecent&size=250&page=1&q=a%20quevedo&ui-citation-summary=true"


In [24]:
total_pages = get_total_pages_from_inspire(driver, url)
papers_per_page = get_papers_per_page(url)

https://inspirehep.net/literature?sort=mostrecent&size=250&q=a%20quevedo&ui-citation-summary=true


In [87]:
df = scrape_all_pages(driver, url, total_pages)

In [90]:
# Set the display option to show full content of the column
pd.set_option('display.max_colwidth', None)
df.head()

,Title,Authors,ePrint Number,arXiv Category,DOI,Citation Count
0,Quasi-periodic oscillations in rotating and deformed spacetimes,"Kuantay Boshkayev, Talgar Konysbayev, Yergali Kurmanov, Marco Muccino, Hernando Quevedo",2312.03630,astro-ph.HE,None,0
1,Extended black hole geometrothermodynamics,Hernando Quevedo,2312.01535,gr-qc,None,0
2,Euclid preparation. TBD. Galaxy power spectrum modelling in real space,A. Pezzotta,2312.00679,astro-ph.CO,None,0
3,Gravitational repulsive effects in 3D regular black holes,"Orlando Luongo, Hernando Quevedo, S.N. Sajadi",2311.13264,gr-qc,None,0
4,Moduli Stabilization in String Theory,"Liam McAllister, Fernando Quevedo",2310.20559,hep-th,None,2


In [97]:
import time
%timeit

# Assuming 'df' is your DataFrame and it has a column named 'ePrint Number'
df['Abstract'] = df['ePrint Number'].apply(fetch_abstract)

In [98]:
df

,Title,Authors,ePrint Number,arXiv Category,DOI,Citation Count,Abstract
0,Quasi-periodic oscillations in rotating and deformed spacetimes,"Kuantay Boshkayev, Talgar Konysbayev, Yergali Kurmanov, Marco Muccino, Hernando Quevedo",2312.03630,astro-ph.HE,None,0,"Quasi-periodic oscillation (QPOs) analysis is important for understanding the dynamical behavior of many astrophysical objects during transient events such as gamma-ray bursts, solar flares, magnetar flares, and fast radio bursts. In this paper, we analyze QPO data in low-mass X-ray binary (LMXB) systems, using the Lense-Thirring, Kerr, and approximate Zipoy-Voorhees metrics. We demonstrate that the inclusion of spin and quadrupole parameters modifies the well-established results for the fundamental frequencies in the Schwarzschild spacetime. We interpret the QPO data within the framework of the standard relativistic precession model, allowing us to infer the values of the mass, spin, and quadrupole parameters of neutron stars in LMXBs. We explore recent QPO data sets from eight distinct LMXBs, assess their optimal parameters, and compare our findings with results in the existing literature. Finally, we discuss the astrophysical implications of our findings."
1,Extended black hole geometrothermodynamics,Hernando Quevedo,2312.01535,gr-qc,None,0,"Although ordinary laboratory thermodynamic systems are known to be homogeneous systems, black holes are different and cannot be considered within this class. Using the formalism of geometrothermodynamics, we show that black holes should be considered as quasi-homogeneous systems. As a consequence, we argue that coupling constants in generalized gravity theories should be considered as thermodynamic variables, giving raise to extended versions of black hole thermodynamics."
2,Euclid preparation. TBD. Galaxy power spectrum modelling in real space,A. Pezzotta,2312.00679,astro-ph.CO,None,0,"We investigate the accuracy of the perturbative galaxy bias expansion in view of the forthcoming analysis of the Euclid spectroscopic galaxy samples. We compare the performance of an Eulerian galaxy bias expansion, using state-of-art prescriptions from the effective field theory of large-scale structure (EFTofLSS), against a hybrid approach based on Lagrangian perturbation theory and high-resolution simulations. These models are benchmarked against comoving snapshots of the Flagship I N-body simulation at $z=(0.9,1.2,1.5,1.8)$, which have been populated with H$\alpha$ galaxies leading to catalogues of millions of objects within a volume of about $58\,h^{-3}\,{\rm Gpc}^3$. Our analysis suggests that both models can be used to provide a robust inference of the parameters $(h, \omega_{\rm c})$ in the redshift range under consideration, with comparable constraining power. We additionally determine the range of validity of the EFTofLSS model in terms of scale cuts and model degrees of freedom. From these tests, it emerges that the standard third-order Eulerian bias expansion can accurately describe the full shape of the real-space galaxy power spectrum up to the maximum wavenumber $k_{\rm max}=0.45\,h\,{\rm Mpc}^{-1}$, even with a measurement precision well below the percent level. In particular, this is true for a configuration with six free nuisance parameters, including local and non-local bias parameters, a matter counterterm, and a correction to the shot-noise contribution. Fixing either tidal bias parameters to physically-motivated relations still leads to unbiased cosmological constraints. We finally repeat our analysis assuming a volume that matches the expected footprint of Euclid, but without considering observational effects, as purity and completeness, showing that we can get consistent cosmological constraints over this range of scales and redshifts."
3,Gravitational repulsive effects in 3D regular black holes,"Orlando Luongo, Hernando Quevedo, S.N. Sajadi",2311.13264,gr-qc,None,0,"In this work, we consider the effects of repulsive gravity in